In [2]:
!pip install transformers datasets sklearn numpy torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.9 MB/s 
     |████████████████████████████████| 365 kB 53.5 MB/s 
     |████████████████████████████████| 596 kB 53.1 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 141 kB 58.8 MB/s 
     |████████████████████████████████| 212 kB 50.9 MB/s 
     |████████████████████████████████| 115 kB 78.5 MB/s 
     |████████████████████████████████| 127 kB 56.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from datasets import load_dataset, load_metric

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving hateSpeechTest.csv to hateSpeechTest.csv
Saving hateSpeechTrain.csv to hateSpeechTrain.csv
User uploaded file "hateSpeechTest.csv" with length 1627010 bytes
User uploaded file "hateSpeechTrain.csv" with length 6454421 bytes


In [5]:
with open("hateSpeechTest.csv",'r') as f:
    with open("hateSpeechTestHeadless.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

In [6]:
files.download('hateSpeechTestHeadless.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
with open("hateSpeechTrain.csv",'r') as f:
    with open("hateSpeechTrainHeadless.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

In [8]:
files.download('hateSpeechTrainHeadless.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
dataset = load_dataset(
    'csv',
    data_files={
        'train': 'hateSpeechTrainHeadless.csv',
        'test': 'hateSpeechTestHeadless.csv'
    },
    column_names = ['sentence', 'label']
)

Using custom data configuration default-d149e7ef57b1d8a0


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d149e7ef57b1d8a0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
len(dataset['train'])

62540

In [11]:
len(dataset['test'])

15634

In [14]:
metric = load_metric('glue', 'sst2')

In [15]:
from transformers import AutoTokenizer

In [16]:
tokenizer = AutoTokenizer.from_pretrained(
    'EMBEDDIA/crosloengual-bert',
    use_fast=True
)

In [17]:
label2id = {'3 nasilje': 3, '2 žalitev': 2, '1 nespodobni govor': 1, '0 ni sporni govor': 0}
id2label = {'0 ni sporni govor', '1 nespodobni govor', '2 žalitev', '3 nasilje'}

In [18]:
def preprocess(examples):
  result = tokenizer(examples['sentence'], truncation=True, max_length=512)
  result['label'] = [label2id[l] for l in examples['label']]
  return result

In [19]:
encoded_dataset = dataset.map(preprocess, batched=True, load_from_cache_file=False)

  0%|          | 0/63 [00:00<?, ?ba/s]

TypeError: ignored